## api test 

In [36]:
import requests
import pandas as pd
import json
from datetime import datetime
from requests.auth import HTTPBasicAuth
import concurrent.futures
import time
import re
from tqdm import tqdm

# Set your client_id and client_secret
client_id = 'TsH-x5Hf'
client_secret = 'YR_pRWYuCL91j6Yj9MQpzr8QSO_zO8ZoOrZ2CQjXF2A'

def get_auth_token():
    """Authenticate to Deribit and return an access token."""
    url = "https://test.deribit.com/api/v2/public/auth"
    payload = {
        "jsonrpc": "2.0",
        "id": 9929,
        "method": "public/auth",
        "params": {
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret
        }
    }
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=payload, headers=headers)
    
    if response.status_code == 200:
        return response.json().get('result', {}).get('access_token')
    else:
        print(f"Authentication failed: {response.status_code}, {response.json()}")
        return None

def get_option_name_and_settlement(coin, token):
    """Retrieve option names and settlement periods with authentication."""
    headers = {"Authorization": f"Bearer {token}"}
    url = f"https://test.deribit.com/api/v2/public/get_instruments?currency={coin}&kind=option"
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        result = response.json()
        name = pd.json_normalize(result['result'])['instrument_name']
        settlement_period = pd.json_normalize(result['result'])['settlement_period']
        
        # Print unique settlement periods
        unique_periods = set(settlement_period)
        print("Unique settlement periods available:", unique_periods)
        
        return list(name), list(settlement_period)
    else:
        print("Failed to fetch data:", response.status_code)
        return None, None

def extract_details(instrument_name, coin):
    """
    Extract expiration date, strike price, and option type (call or put) from instrument name.
    Adjusts to include selected coin (e.g., 'BTC' or 'ETH').
    """
    match = re.match(fr"{coin}-(\d+[A-Z]{{3}}\d+)-(\d+)-([CP])", instrument_name)
    if match:
        expiration_date = match.group(1)
        strike_price = match.group(2)
        option_type = 'Call' if match.group(3) == 'C' else 'Put'
        return expiration_date, strike_price, option_type
    return None, None, None

def fetch_option_data(option_name, token):
    """Fetch the option data for a given option name with authentication."""
    time.sleep(0.1)  # Add a short delay to avoid rate limits
    headers = {"Authorization": f"Bearer {token}"}
    url = f'https://test.deribit.com/api/v2/public/get_order_book?instrument_name={option_name}'
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        result = response.json()
        df = pd.json_normalize(result['result'])
        selected_columns = ["instrument_name", "mark_price", "underlying_price", "mark_iv", "greeks.vega"]
        return df[selected_columns]
    else:
        print(f"Failed to fetch option data for {option_name}: {response.status_code}")
        return None

def get_option_data(coin, settlement_per):
    """Main function to get and process options data."""
    # Authenticate to get token
    token = get_auth_token()
    if not token:
        print("Token retrieval failed.")
        return None
    
    # Get option names and settlement
    coin_name, settlement_period = get_option_name_and_settlement(coin, token)
    
    # Check if settlement_per is actually present
    if settlement_per not in settlement_period:
        print(f"No options available with settlement period '{settlement_per}'.")
        return None
    
    # Filter options that match settlement_per
    coin_name_filtered = [coin_name[i] for i in range(len(coin_name)) if settlement_period[i] == settlement_per]
    print("Filtered option names:", coin_name_filtered)  # Print filtered option names
    
    # Initialize progress bar
    pbar = tqdm(total=len(coin_name_filtered))
    
    # Fetch data concurrently using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_option = {executor.submit(fetch_option_data, name, token): name for name in coin_name_filtered}
        coin_df = []
        for future in concurrent.futures.as_completed(future_to_option):
            try:
                data = future.result()
                if data is not None:
                    data['settlement_period'] = settlement_per
                    coin_df.append(data)
            except Exception as exc:
                print(f'Error fetching data: {exc}')
            pbar.update(1)
    
    # Concatenate list of DataFrames if coin_df is not empty
    if coin_df:
        coin_df = pd.concat(coin_df, ignore_index=True)
    else:
        print("No data fetched.")
        return None
    
    # Extract expiration date, strike price, and option type
    coin_df['Expiration Date'], coin_df['Strike Price'], coin_df['Option Type'] = zip(*coin_df['instrument_name'].apply(lambda x: extract_details(x, coin)))
    
    # Calculate Time to Expiration
    today = datetime.today()
    coin_df['Time to Expiration'] = coin_df['Expiration Date'].apply(lambda x: (datetime.strptime(x, '%d%b%y') - today).days / 365 if x else None)
    
    # Select final columns
    final_columns = ["instrument_name", "Option Type", "Expiration Date", "Strike Price", 'Time to Expiration', 'mark_price', 'underlying_price', 'mark_iv', 'greeks.vega', 'settlement_period']
    coin_df = coin_df[final_columns]
    pbar.close()
    
    return coin_df

# Main execution
if __name__ == "__main__":
    coin = "BTC"
    settlement_per = "month"  # Adjust based on actual available values
    
    # Save the data to a variable named "data"
    data = get_option_data(coin, settlement_per)
    if data is not None:
        print("Option data successfully retrieved and saved to variable 'data'.")
        print(data.head())  # Display the first few rows of data
    else:
        print("Failed to retrieve option data.")


Unique settlement periods available: {'month', 'week', 'day'}
Filtered option names: ['BTC-29NOV24-32000-C', 'BTC-29NOV24-32000-P', 'BTC-29NOV24-34000-C', 'BTC-29NOV24-34000-P', 'BTC-29NOV24-36000-C', 'BTC-29NOV24-36000-P', 'BTC-29NOV24-38000-C', 'BTC-29NOV24-38000-P', 'BTC-29NOV24-40000-C', 'BTC-29NOV24-40000-P', 'BTC-29NOV24-42000-C', 'BTC-29NOV24-42000-P', 'BTC-29NOV24-43000-C', 'BTC-29NOV24-43000-P', 'BTC-29NOV24-44000-C', 'BTC-29NOV24-44000-P', 'BTC-29NOV24-45000-C', 'BTC-29NOV24-45000-P', 'BTC-29NOV24-46000-C', 'BTC-29NOV24-46000-P', 'BTC-29NOV24-47000-C', 'BTC-29NOV24-47000-P', 'BTC-29NOV24-48000-C', 'BTC-29NOV24-48000-P', 'BTC-29NOV24-49000-C', 'BTC-29NOV24-49000-P', 'BTC-29NOV24-50000-C', 'BTC-29NOV24-50000-P', 'BTC-29NOV24-51000-C', 'BTC-29NOV24-51000-P', 'BTC-29NOV24-52000-C', 'BTC-29NOV24-52000-P', 'BTC-29NOV24-53000-C', 'BTC-29NOV24-53000-P', 'BTC-29NOV24-54000-C', 'BTC-29NOV24-54000-P', 'BTC-29NOV24-55000-C', 'BTC-29NOV24-55000-P', 'BTC-29NOV24-56000-C', 'BTC-29NOV24-5600

100%|██████████| 634/634 [01:21<00:00,  7.82it/s]

Option data successfully retrieved and saved to variable 'data'.
       instrument_name Option Type Expiration Date Strike Price  \
0  BTC-29NOV24-44000-C        Call         29NOV24        44000   
1  BTC-29NOV24-44000-P         Put         29NOV24        44000   
2  BTC-29NOV24-36000-C        Call         29NOV24        36000   
3  BTC-29NOV24-36000-P         Put         29NOV24        36000   
4  BTC-29NOV24-40000-C        Call         29NOV24        40000   

   Time to Expiration  mark_price  underlying_price  mark_iv  greeks.vega  \
0            0.079452      0.4004          73250.63    84.49      6.52308   
1            0.079452      0.0011          73250.63    84.49      6.54330   
2            0.079452      0.5088          73250.63    94.67      1.85325   
3            0.079452      0.0003          73250.63    94.67      1.85325   
4            0.079452      0.4547          73250.63    93.55      4.41884   

  settlement_period  
0             month  
1             month  
2  

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   instrument_name     634 non-null    object 
 1   Option Type         634 non-null    object 
 2   Expiration Date     634 non-null    object 
 3   Strike Price        634 non-null    object 
 4   Time to Expiration  634 non-null    float64
 5   mark_price          634 non-null    float64
 6   underlying_price    634 non-null    float64
 7   mark_iv             634 non-null    float64
 8   greeks.vega         634 non-null    float64
 9   settlement_period   634 non-null    object 
dtypes: float64(5), object(5)
memory usage: 49.7+ KB
